# Source estimation

In [ ]:
%matplotlib qt

In [ ]:
import pathlib
import matplotlib
import matplotlib.pyplot as plt
import mne_bids
import mne

matplotlib.use('Qt5Agg')
mne.set_log_level('warning')

## Start with some fresh epochs

In [ ]:
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq=0.1, h_freq=40)
events, event_id = mne.events_from_annotations(raw)

tmin = -0.200
tmax = 0.500
baseline = (None, 0)

epochs = mne.Epochs(raw,
                    events=events,
                    event_id=event_id,
                    tmin=tmin,
                    tmax=tmax,
                    baseline=baseline,
                    preload=True,
                    proj=False)
epochs.save(pathlib.Path('out_data') / 'epochs_for_source_epo.fif')
epochs.info

## View the BEM

In [ ]:
subjects_dir = pathlib.Path(mne.datasets.sample.data_path()) / 'subjects'

mne.viz.plot_bem(subject='sample', subjects_dir=subjects_dir,
                 orientation='coronal')

## Coregistration!

In [ ]:
epochs_fname = pathlib.Path('out_data') / 'epochs_for_source_epo.fif'

mne.gui.coregistration(subject='sample', subjects_dir=subjects_dir,
                       inst=epochs_fname)

In [ ]:
trans_fname = pathlib.Path('out_data') / 'sample.trans'
info = mne.io.read_info(epochs_fname)
fig = mne.viz.plot_alignment(info=info, trans=trans_fname, subject='sample', dig=True,
                             subjects_dir=subjects_dir, verbose=True)

## Compute the source space

In [ ]:
subject = 'sample'
src = mne.setup_source_space(subject=subject,
                             spacing='oct4',  # Use oct6 during an actual analysis!
                             subjects_dir=subjects_dir,
                             add_dist=False)  # Remove this one during an actual analysis!
src

In [ ]:
mne.viz.plot_alignment(info=info, trans=trans_fname, subject=subject,
                       src=src, subjects_dir=subjects_dir, dig=True,
                       surfaces=['head-dense', 'white'], coord_frame='meg')

## Compute the forward solution!

In [ ]:
conductivity = (0.3,)  # for single layer – used in MEG
# conductivity = (0.3, 0.006, 0.3)  # for three layers – used in EEG
model = mne.make_bem_model(subject=subject, ico=4,
                           conductivity=conductivity,
                           subjects_dir=subjects_dir)
model

In [ ]:
bem_sol = mne.make_bem_solution(model)
bem_sol

In [ ]:
bem_fname = pathlib.Path('out_data') / 'sample_bem.fif'
mne.bem.write_bem_solution(bem_fname, bem_sol)

In [ ]:
fwd = mne.make_forward_solution(raw_fname,
                                trans=trans_fname,
                                src=src,
                                bem=bem,
                                meg=True, # include MEG channels
                                eeg=False, # exclude EEG channels
                                mindist=5.0, # ignore sources <= 5mm from inner skull
                                n_jobs=1) # number of jobs to run in parallel
fwd

In [ ]:
fwd_fname = pathlib.Path('out_data') / 'sample_fwd.fif'
mne.write_forward_solution(fwd_fname, fwd, overwrite=True)

## Compute noise covariance

In [ ]:
noise_cov = mne.compute_covariance(epochs, tmax=0.,
                                   method=['shrunk', 'empirical'],
                                   rank='info')
mne.viz.plot_cov(noise_cov, info=info)

In [ ]:
contrast.plot_white(noise_cov)

## Create the inverse operator

In [ ]:
from mne.forward import read_forward_solution
from mne.minimum_norm import (make_inverse_operator, apply_inverse,
                              write_inverse_operator)

In [ ]:
fwd_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01-meg-fwd.fif')
fwd = mne.read_forward_solution(fwd_fname)
fwd = mne.convert_forward_solution(fwd, surf_ori=True)

# Restrict forward solution as necessary for MEG
fwd = mne.pick_types_forward(fwd, meg=True, eeg=False)

# make an M/EEG, MEG-only, and EEG-only inverse operator
info = contrast.info
inverse_operator = make_inverse_operator(info, fwd, noise_cov,
                                         loose=0.2, depth=0.8)

## Apply the inverse operator -> Calculate the source estimate

In [ ]:
method = "dSPM"
snr = 3.
lambda2 = 1. / snr ** 2
stc = apply_inverse(epochs['Auditory/Left', inverse_operator, lambda2,
                    method=method, pick_ori=None)

## Show the result!

In [ ]:
brain = stc.plot(surface='inflated',
                 hemi='both',
                 subjects_dir=subjects_dir,
                 time_viewer=True)